In [23]:
from typing import List, Union, Tuple, Dict, TypedDict
from IPython.display import clear_output

In [24]:
class ColumnFullException(Exception):
    pass

In [25]:
class ConnectFour:
    board_height = 6
    columns = ["A", "B", "C", "D", "E", "F", "G"]
    board = {}
    player_turn = 1
    
    def set_up_board(self):
        for column in self.columns:
            self.board[column] = [None for i in range(self.board_height)]
    
    def display_board(self) -> None:
        players = ["◯", "\x1b[31m●\x1b[0m", "●"]
        output = " ┌┌––––––––––––––––––––┐┐\n"
        for i in range(self.board_height-1, -1, -1):
            output += " |"
            for column, row in self.board.items():
                slot = players[row[i]] if row[i] is not None else "_"
                output += f"|{slot} "
            output += "||\n"
        output += " |┕━━━━━━━━━━━━━━━━━━━━┙|\n"
        output += "╱╱ A  B  C  D  E  F  G  ╲╲ "
        print(output)
    
    def set_piece(self, column: str, player: int) -> int:
        # returns the index of where the piece landed
        # if all 6 rows are occupied let the use know
        selected_column = self.board[column]
        for idx, slot in enumerate(selected_column):
            if slot is None:
                selected_column[idx] = player
                return idx
        raise ColumnFullException
        
    def did_player_win(self, player: int, column_index: int, row_index: int) -> bool:
        row_group = self._get_row_group(row_index)
        column_group = self._get_column_group(column_index)
        diagonal_down = self._get_descending_diagonal_group(column_index, row_index)
        diagonal_up = self._get_ascending_diagonal_group(column_index, row_index)
        ways_to_win = (row_group, column_group, diagonal_down, diagonal_up)

        for group in ways_to_win:
            if self._check_result(player, group):
                return True
        return False
    
    def _check_result(self, player: int, piece_group: List) -> bool:
        slot_streak = 0
        for slot in piece_group:
            if slot == player:
                slot_streak += 1
                if slot_streak == 4:
                    return True
            else:
                slot_streak = 0            
        return False

    def _get_row_group(self, row_index: int) -> List[int]:
        return [column[row_index] for column in self.board.values()]
    
    def _get_column_group(self, column_key: str) -> List[int]:
        return self.board.get(column_key)
    
    def _get_descending_diagonal_group(self, column_key: str, row_index: int) -> List[int]:
        diagonal_desc = []
        columns = list(self.board.keys())

        # considering the row index and column,
        # climb up and to the left to find the start of the run
        group_column_index = columns.index(column_key)
        group_row_index = row_index
        while group_column_index > 0 and group_row_index < self.board_height - 1:
            group_column_index -= 1
            group_row_index += 1
        for col in columns[group_column_index:]:
            diagonal_desc.append(self.board[col][group_row_index])
            if group_row_index == 0:
                break
            group_row_index -= 1
        
        return diagonal_desc
        
    
    def _get_ascending_diagonal_group(self, column_key: str, row_index: int) -> List[int]:
        diagonal_asc = []
        columns = list(self.board.keys())
        
        # climb down and to the left to find the start of the run
        group_column_index = columns.index(column_key)
        group_row_index = row_index
        while group_column_index > 0 and group_row_index > 0:
            group_column_index -= 1
            group_row_index -= 1
        for col in columns[group_column_index:]:
            diagonal_asc.append(self.board[col][group_row_index])
            if group_row_index == self.board_height - 1:
                break
            group_row_index += 1
        return diagonal_asc
    
    def switch_player_turns(self):
        if self.player_turn == 1:
            self.player_turn = 2
        else:
            self.player_turn = 1
    
    def play(self) -> None:
        print(f"It's player {self.player_turn}'s turn.")
        self.display_board()
        player_selected_column = input().upper()
        try:
            piece_row = self.set_piece(player_selected_column, self.player_turn)
        except KeyError:
            clear_output()
            print(f"Invalid column selection: {player_selected_column}")
            self.play()
        except ColumnFullException:
            clear_output()
            print(f"Column {player_selected_column} is full, select a different column.")
            self.play()
        if not self.did_player_win(self.player_turn, player_selected_column, piece_row):
            self.switch_player_turns()
            clear_output()
            self.play()
        else:
            clear_output()
            self.display_board()
            print(f"Congratulations!  Player {self.player_turn} is the winner!")
            
    def start(self):
        self.set_up_board()
        self.play()

In [26]:
cf = ConnectFour()
cf.start()

It's player 2's turn.
 ┌┌––––––––––––––––––––┐┐
 ||_ |_ |_ |_ |_ |_ |_ ||
 ||_ |_ |_ |_ |_ |_ |_ ||
 ||_ |_ |_ |_ |_ |_ |_ ||
 ||_ |_ |_ |_ |_ |_ |_ ||
 ||_ |_ |_ |_ |_ |_ |_ ||
 ||_ |● |● |● |_ |_ |_ ||
 |┕━━━━━━━━━━━━━━━━━━━━┙|
╱╱ A  B  C  D  E  F  G  ╲╲ 


KeyboardInterrupt: Interrupted by user

In [22]:
cf.board

{'A': [None, None, None, None, None, None],
 'B': [None, None, None, None, None, None],
 'C': [None, None, None, None, None, None],
 'D': [None, None, None, None, None, None],
 'E': [None, None, None, None, None, None],
 'F': [None, None, None, None, None, None],
 'G': [None, None, None, None, None, None]}

In [6]:
cf.board

{'A': [2, None, None, None, None, None],
 'B': [1, None, None, None, None, None],
 'C': [2, 2, None, None, None, None],
 'D': [2, 2, None, None, None, None],
 'E': [1, 2, None, None, None, None],
 'F': [1, 2, 1, 1, 1, None],
 'G': [1, 1, 2, None, None, None]}

In [368]:
save_board = cf.board.copy()

In [11]:
column = [None, None, None, None, None, None]
board = []
for i in range(7):
    board.append(list(column))

In [12]:
def set_piece(column:int, player: int):
    # a player inputs a int where the piece falls,
    # the piece falls to the lowest unoccupied row
    # if all 6 rows are occupied let the use know
    selected_column = board[column]
    slot_filled = False
    for idx, slot in enumerate(selected_column):
        if slot_filled:
            break
        if slot is None and slot_filled is False:
            selected_column[idx] = player
            slot_filled = True

    if not slot_filled:
        raise ColumnFullException
    

        


In [13]:
def who_won(group: List, player:int) -> Union[int, None]:
    slot_streak = 0
    for slot in group:
        # looks for the same player int, 4 times in a row
        # returns player int, if that player won
        # returns none if game is not over
        if slot == player:
            slot_streak += 1
            if slot_streak > 3:
                return player
        else:
            slot_streak = 0

def check_result(column_index: int, row_index: int) -> Union[int, None]:
    # check_horizontal = same index each column
    # check_vertical = same column
    # diagonal_desc = over adjacent columns (+1 row to left, -1 row to right)
    # diagonal_asc = over adjacent cols (-1 row to left, +1 row to right)
    column_group = board[column_index]
    player = column_group[row_index]
    if who_won(column_group, player):
        return player
    row_group = [column[row_index] for column in board]
    if who_won(row_group, player):
        return player
    
    diagonal_desc = []
    left_col, left_row = column_index, row_index
    while left_col >= 0 and left_row < len(column_group) - 1:
        diagonal_desc.append(board[left_col][left_row])
        left_col -= 1
        left_row += 1
    right_col, right_row = column_index, row_index
    while right_col < len(board) - 1 and right_row >= 0:
        right_col += 1
        right_row -= 1
        if right_row < 0 or right_col == len(board):
            break
        diagonal_desc.append(board[right_col][right_row])
    if who_won(diagonal_desc, player):
        return player
    
    diagonal_asc = []
    left_col, left_row = column_index, row_index
    while left_col < len(column_group) - 1 and left_row >= 0:
        diagonal_desc.append(board[left_col][left_row])
        left_col += 1
        left_row -= 1
    right_col, right_row = column_index, row_index
    while right_col < len(board) - 1 and right_row >= 0:
        right_col += 1
        right_row -= 1
        if right_row < 0 or right_col == len(board):
            break
        diagonal_desc.append(board[right_col][right_row])

def is_game_over(board: List[List]) -> bool:
    # look at the board and check for a win conditon from either player
    # 4 of the same player-pieces in a row, vertically , hor, diagonally
    
    # vertical
    for column in board:
        pass
    # horizontal 
    # take the same row# and check each column at that row
    
        

In [14]:
set_piece(3, 2)

In [15]:
board

[[None, None, None, None, None, None],
 [None, None, None, None, None, None],
 [None, None, None, None, None, None],
 [2, None, None, None, None, None],
 [None, None, None, None, None, None],
 [None, None, None, None, None, None],
 [None, None, None, None, None, None]]

In [243]:
winner = check_result(3, 0)

In [244]:
print(winner)

2


In [27]:
import json

In [28]:
class BoardState(TypedDict):
    player_turn: int
    player_selected_column: int
    board: Dict["str", List[Union[int, None]]]

In [29]:
bs = BoardState(
    board=cf.board,
    player_turn=2,
)

In [34]:
bs.get("player_turn")

2

In [30]:
board_string = json.dumps(bs)

In [31]:
board_string

'{"board": {"A": [null, null, null, null, null, null], "B": [2, null, null, null, null, null], "C": [1, null, null, null, null, null], "D": [1, null, null, null, null, null], "E": [null, null, null, null, null, null], "F": [null, null, null, null, null, null], "G": [null, null, null, null, null, null]}, "player_turn": 2}'

In [20]:
bs2 = BoardState(json.loads(board_string))

In [21]:
bs2

{'board': {'A': [None, None, None, None, None, None],
  'B': [None, None, None, None, None, None],
  'C': [None, None, None, None, None, None],
  'D': [None, None, None, None, None, None],
  'E': [None, None, None, None, None, None],
  'F': [None, None, None, None, None, None],
  'G': [None, None, None, None, None, None]},
 'player_turn': 1}

In [15]:
cx = ConnectFour()

In [16]:
cx.set_up_board()

In [17]:
cx.board

{'A': [None, None, None, None, None, None],
 'B': [None, None, None, None, None, None],
 'C': [None, None, None, None, None, None],
 'D': [None, None, None, None, None, None],
 'E': [None, None, None, None, None, None],
 'F': [None, None, None, None, None, None],
 'G': [None, None, None, None, None, None]}

In [ ]:
cx.b